In this blog we will talk about how to install Pixtral on a GPU cloud. Pixtral is a multimodel with exceptional vision capabilities.

Checkout this blog in video [![Installing Pixtral(on Runpod)](https://img.youtube.com/vi/ndjMi8p9uuw/0.jpg)](https://www.youtube.com/watch?v=ndjMi8p9uuw "Installing Pixtral(on Runpod)")


First we begin with installing VLLM. [VLLM](https://github.com/vllm-project/vllm) is a high-throughput and memory-efficient inference and serving engine for LLMs. We will use VLLM to serve Pixtral. 

In [1]:
!pip install --upgrade vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 MB 54.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 16.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 138.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 82.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 153.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 27.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 135.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7

Next, we install `mistral_common` library. This library will allow us to work with Pixtral. 


In [2]:
!pip install --upgrade mistral_common

Now, we need to get the HuggingFace tokens, in order to download the model. 


In [3]:
import getpass

hf_token = getpass.getpass('Enter your HF Token:')

We authenticate with HuggingFace using the provided token.

In [4]:
from huggingface_hub import login

login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Now we download the model from HuggingFace and run a test inference to check if everything is working fine. 


In [5]:
from vllm import LLM
from vllm.sampling_params import SamplingParams

model_name = "mistralai/Pixtral-12B-2409"

sampling_params = SamplingParams(max_tokens=8192)

llm = LLM(model=model_name, tokenizer_mode="mistral", )

prompt = "Describe this image in one sentence."
image_url = "https://picsum.photos/id/237/200/300"

messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": prompt}, {"type": "image_url", "image_url": {"url": image_url}}]
    },
]

outputs = llm.chat(messages, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)

2024-09-19 15:00:12,682	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


params.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

INFO 09-19 15:00:13 config.py:1653] Downcasting torch.float32 to torch.float16.
WARNING 09-19 15:00:13 arg_utils.py:910] The model has a long context length (128000). This may cause OOM errors during the initial memory profiling phase, or result in low performance due to small KV cache space. Consider setting --max-model-len to a smaller value.
INFO 09-19 15:00:13 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='mistralai/Pixtral-12B-2409', speculative_config=None, tokenizer='mistralai/Pixtral-12B-2409', skip_tokenizer_init=False, tokenizer_mode=mistral, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=128000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=

tekken.json:   0%|          | 0.00/19.3M [00:00<?, ?B/s]

INFO 09-19 15:00:16 model_runner.py:997] Starting to load model mistralai/Pixtral-12B-2409...


...

INFO 09-19 15:03:14 weight_utils.py:287] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-19 15:03:48 model_runner.py:1008] Loading model weights took 23.6552 GB
INFO 09-19 15:04:01 gpu_executor.py:122] # GPU blocks: 12441, # CPU blocks: 1638
INFO 09-19 15:04:05 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-19 15:04:05 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-19 15:04:16 model_runner.py:1430] Graph capturing finished in 12 secs.


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.15it/s, est. speed input: 598.02 toks/s, output: 51.81 toks/s]

The image depicts a black dog lying down on a wooden floor, looking directly at the camera with a relaxed expression.


Another inference for fun 😁

In [15]:
prompt = "Describe this image in detail."
image_url = "https://99designs-blog.imgix.net/blog/wp-content/uploads/2019/03/0-e1552926315420.jpg?auto=format&q=60&w=1693&h=1494&fit=crop&crop=faces"

messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": prompt}, {"type": "image_url", "image_url": {"url": image_url}}]
    },
]

outputs = llm.chat(messages, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.03s/it, est. speed input: 1226.51 toks/s, output: 55.80 toks/s]

The image is a black and white illustration that focuses on the theme of lettering. The title, "The Basics of Lettering", is prominently displayed in the center of the image. The text is written in a variety of styles and sizes, creating a dynamic and visually interesting composition. Some of the letters are cursive, while others are more angular, adding to the diversity of the lettering styles. The letters are arranged in a hand-lettered style, giving the image a personal and artistic feel. The background is white, which contrasts with the black text and allows the lettering to stand out. Scattered throughout the image are small black dots, adding an element of texture and depth to the overall design. The image seems to be a visual guide or tutorial on the basics of lettering, as suggested by the title.


Conclusion: In this blog we learned how to install Pixtral on a GPU cloud. We also learned how to use VLLM to serve Pixtral.